In [1]:
import tensorflow as tf
import coremltools as ct

from datetime import datetime, timedelta
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

print(tf.__version__)

2.3.1


In [2]:
def normalize(x):
    x = 2 * ( (x - np.min(x)) / (np.max(x) - np.min(x)) ) - 1
    return x


def demean(x):
    x = x - np.mean(x)
    return x

In [3]:
clss = [0,1,2,4] # mask_off, normal_breathing, talking, cough
X_train, Y_train, X_test, Y_test = [], [], [], []


data_files = []
for root, subdirs, files in os.walk('data/train'):
    if len(files) == 0: continue
    
    for file in files:
        data_files.append(f'{root.split("/")[-1]}/{file}')

print(data_files)


for file in data_files:
    train = np.load(f'data/train/{file}')
    test = np.load(f'data/test/{file}')

    # normalize sample
    for s in train:
        sample = np.vstack((
                demean(s[0]),
                demean(s[1])
            ))
        X_train.append(sample)
        if 'mask_off' in file: Y_train.append(0)
        elif 'normal_breathing' in file: Y_train.append(1)
        elif 'talking' in file: Y_train.append(2)
        elif 'cough' in file: Y_train.append(3)

    for s in test:
        X_test.append(
            np.vstack((
                demean(s[0]),
                demean(s[1])
            ))
        )
        if 'mask_off' in file: Y_test.append(0)
        elif 'normal_breathing' in file: Y_test.append(1)
        elif 'talking' in file: Y_test.append(2)
        elif 'cough' in file: Y_test.append(3)
    
X_train, Y_train, X_test, Y_test = np.array(X_train, dtype='float64'), np.array(Y_train, dtype='float64'), np.array(X_test, dtype='float64'), np.array(Y_test, dtype='float64')

['talking/s_2021-02-11T01:37:39.912Z.npy', 'talking/s_2021-02-11T01:55:17.151Z.npy', 'talking/s_2021-02-11T02:16:45.294Z.npy', 'talking/s_2021-02-10 18:34:39.763.npy', 'talking/s_2021-02-11T02:17:25.710Z.npy', 'talking/s_2021-02-10 16:36:38.417.npy', 'talking/s_2021-02-11T02:18:05.753Z.npy', 'talking/s_2021-02-11T01:53:39.682Z.npy', 'talking/s_2021-02-11T02:16:02.647Z.npy', 'talking/s_2021-02-11T01:38:19.935Z.npy', 'talking/s_2021-02-11T02:15:23.053Z.npy', 'talking/s_2021-02-11T01:54:29.064Z.npy', 'cough/s_2021-02-10 22:18:44.719.npy', 'cough/s_2021-02-10 20:53:33.741.npy', 'cough/s_2021-02-10 22:18:17.501.npy', 'cough/s_2021-02-10 20:57:59.617.npy', 'normal_breathing/s_2021-02-11T02:10:12.422Z.npy', 'normal_breathing/s_2021-02-11T02:10:52.762Z.npy', 'normal_breathing/s_2021-02-10 15:11:17.966.npy', 'normal_breathing/s_2021-02-11T02:08:10.624Z.npy', 'normal_breathing/s_2021-02-10 18:12:14.117.npy', 'normal_breathing/s_2021-02-11T02:08:51.770Z.npy', 'normal_breathing/s_2021-02-11T01:31:

In [28]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(2,51), name='input'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(4, name='classification')
])
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True ),
    metrics=['accuracy']
)

In [29]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Flatten)              (None, 102)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                3296      
_________________________________________________________________
dense_11 (Dense)             (None, 16)                528       
_________________________________________________________________
classification (Dense)       (None, 4)                 68        
Total params: 3,892
Trainable params: 3,892
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.fit(np.nan_to_num(X_train), Y_train, epochs=750, batch_size=len(X_train))

Epoch 551/750
1/1 [==============================] - 0s 1ms/step - loss: 0.0563 - accuracy: 0.9869
Epoch 552/750
1/1 [==============================] - 0s 2ms/step - loss: 0.0561 - accuracy: 0.9869
Epoch 553/750
1/1 [==============================] - 0s 1ms/step - loss: 0.0558 - accuracy: 0.9869
Epoch 554/750
1/1 [==============================] - 0s 962us/step - loss: 0.0556 - accuracy: 0.9869
Epoch 555/750
1/1 [==============================] - 0s 2ms/step - loss: 0.0553 - accuracy: 0.9869
Epoch 556/750
1/1 [==============================] - 0s 749us/step - loss: 0.0551 - accuracy: 0.9869
Epoch 557/750
1/1 [==============================] - 0s 825us/step - loss: 0.0548 - accuracy: 0.9869
Epoch 558/750
1/1 [==============================] - 0s 1ms/step - loss: 0.0546 - accuracy: 0.9869
Epoch 559/750
1/1 [==============================] - 0s 770us/step - loss: 0.0543 - accuracy: 0.9875
Epoch 560/750
1/1 [==============================] - 0s 1ms/step - loss: 0.0541 - accuracy: 0.9875
Ep

In [31]:
test_loss, test_acc = model.evaluate(np.nan_to_num(X_test),  Y_test, verbose=2)
print('\nTest accuracy:', test_acc)

25/25 - 0s - loss: 0.3935 - accuracy: 0.9416

Test accuracy: 0.9415584206581116


In [32]:
model.save('simple_nn')
mlmodel = ct.convert('simple_nn') 

Translating MIL ==> MLModel Ops: 100%|██████████| 13/13 [00:00<00:00, 12081.97 ops/s]


In [33]:
mlmodel.author = "Blaine Rothrock"
mlmodel.short_description = "Classification of 3 second temp and pressure data @ 17hz. Classes: Mask Off, Normal Breathing, Talking, Cough. Cough has very little representation"
mlmodel.version = "0.0.1"

In [34]:
mlmodel.save('RespiratoryClassifier.mlmodel')